**Set Up Environment**

In [ ]:
!pip install music21 pretty_midi tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=145e3a8a0639eab7ee75a9c7a0cb4442f62a2f97819d73be562e8d443b53e28d
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


In [ ]:
# Download the ZIP file containing the Nottingham database
!wget http://ifdo.ca/~seymour/nottingham/nottingham_database.zip

# Unzip the dataset
!unzip nottingham_database.zip -d nottingham

# Move into the folder with .mid files
%cd nottingham


URL transformed to HTTPS due to an HSTS policy
--2025-07-10 16:55:35--  https://ifdo.ca/~seymour/nottingham/nottingham_database.zip
Resolving ifdo.ca (ifdo.ca)... 192.95.24.124, 2607:5300:60:287c::1
Connecting to ifdo.ca (ifdo.ca)|192.95.24.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142934 (140K) [application/zip]
Saving to: ‘nottingham_database.zip’

nottingham_database 100%[===================>] 139.58K  --.-KB/s    in 0.09s   

2025-07-10 16:55:35 (1.46 MB/s) - ‘nottingham_database.zip’ saved [142934/142934]

Archive:  nottingham_database.zip
   creating: nottingham/nottingham_database/
  inflating: nottingham/nottingham_database/reelsH-L.abc  
  inflating: nottingham/nottingham_database/ashover.abc  
  inflating: nottingham/nottingham_database/xmas.abc  
  inflating: nottingham/nottingham_database/jigs.abc  
  inflating: nottingham/nottingham_database/nottingham.html  
  inflating: nottingham/nottingham_database/waltzes.abc  
  inflating: nottingh

**Convert .abc to .mid**

In [ ]:
!apt install abcmidi
!mkdir midi_files

# Convert all .abc to .mid
!find . -name "*.abc" -exec sh -c 'for f; do abc2midi "$f" -o "midi_files/$(basename "$f" .abc).mid"; done' sh {} +

# Move into MIDI folder
%cd midi_files


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  abcm2ps timidity | pmidi postscript-viewer
The following NEW packages will be installed:
  abcmidi
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 306 kB of archives.
After this operation, 868 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 abcmidi amd64 20220218+ds1-1 [306 kB]
Fetched 306 kB in 0s (1,149 kB/s)
Selecting previously unselected package abcmidi.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../abcmidi_20220218+ds1-1_amd64.deb ...
Unpacking abcmidi (20220218+ds1-1) ...
Setting up abcmidi (20220218+ds1-1) ...
Processing triggers for man-db (2.10.2-1) ...
4.68 February 18 2022 abc2midi
voice mapping:
  1   
Warning in line-char 15-22 : Assuming repeat
Warning in line-char 15-22 : replacing |: with double repeat (::)
writing MIDI f

In [ ]:
import glob
print("MIDI files found:", len(glob.glob("*.mid")))


MIDI files found: 14


**Extract Notes from MIDI Files**

In [ ]:
from music21 import converter, note, chord
import glob

notes = []

for file in glob.glob("*.mid"):
    try:
        midi = converter.parse(file)
        parts = midi.flat.notes  # No instrument splitting

        for element in parts:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    except Exception as e:
        print(f"Error processing {file}: {e}")

print("Total notes extracted:", len(notes))
print("Sample notes:", notes[:10])


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Total notes extracted: 3950
Sample notes: ['C5', 'B4', 'A4', 'A2', 'G#4', 'A4', '9.0.4', 'B4', 'C5', 'A2']


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


**Prepare Sequences for LSTM**

In [ ]:
sequence_length = 25  # Shorter because of small dataset
pitch_names = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(pitch_names))

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

import numpy as np
from tensorflow.keras.utils import to_categorical

n_patterns = len(network_input)
n_vocab = len(pitch_names)

X = np.reshape(network_input, (n_patterns, sequence_length, 1))
X = X / float(n_vocab)
y = to_categorical(network_output)


**Define and Train the LSTM Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=64)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 247ms/step - loss: 3.7134
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 265ms/step - loss: 3.4979
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 254ms/step - loss: 3.4799
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 259ms/step - loss: 3.5036
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 279ms/step - loss: 3.4664
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 18s 299ms/step - loss: 3.4514
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 257ms/step - loss: 3.4231
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 250ms/step - loss: 3.3698
Epoch 9/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 17s 265ms/step - loss: 3.3037
Epoch 10/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 253ms/step - loss: 3.2497
Epoch 11/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 266ms/step - loss: 3.2385
Epoch 12/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 20s 251ms/step - loss: 3.1874
Epoch 13/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 16s 253ms/step - loss: 3.1752
Epoch 14/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 258ms/step - loss: 3.0424
Epoch 15/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 22s 

**Generate a New Melody**

In [ ]:
int_to_note = dict((number, note) for note, number in note_to_int.items())
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
generated_notes = []

for note_index in range(100):  # Shorter output due to small dataset
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_notes.append(result)
    pattern.append(index)
    pattern = pattern[1:]


**Convert Notes to MIDIt**

In [ ]:
from music21 import stream, note, chord

output_notes = []

for pattern in generated_notes:
    if '.' in pattern or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        chord_notes = [note.Note(int(n)) for n in notes_in_chord]
        new_chord = chord.Chord(chord_notes)
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        output_notes.append(new_note)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')


'generated_music.mid'

In [ ]:
from google.colab import files
files.download('generated_music.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Convert MIDI to MP3 or WAV in Python**

In [ ]:
!apt-get install -y fluidsynth
!pip install pyfluidsynth
!wget https://github.com/urish/cinto/raw/master/soundfonts/FluidR3_GM.sf2


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

**Convert MIDI to WAV**

In [ ]:
import subprocess

midi_file = "generated_music.mid"
output_wav = "generated_music.wav"
soundfont = "FluidR3_GM.sf2"

# Run Fluidsynth command
subprocess.run([
    "fluidsynth", "-ni", soundfont, midi_file, "-F", output_wav, "-r", "44100"
])


CompletedProcess(args=['fluidsynth', '-ni', 'FluidR3_GM.sf2', 'generated_music.mid', '-F', 'generated_music.wav', '-r', '44100'], returncode=0)

**Download or Play the WAV**

In [ ]:
from google.colab import files
files.download("generated_music.wav")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Next Step**

**1.Generate Multi-Track Music**

**Prepare Melody and Chord Sequences from Dataset**

In [ ]:
melody_notes = []
chord_notes = []

for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = midi.flat.notesAndRests.stream()

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            melody_notes.append(str(element.pitch))
            chord_notes.append("None")  # placeholder
        elif isinstance(element, chord.Chord):
            chord_notes.append('.'.join(str(n) for n in element.normalOrder))
            melody_notes.append("None")  # placeholder


/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


**Build Sequences**

In [ ]:
sequence_length = 25

# Melody
melody_input = []
melody_output = []

# Chords
chord_input = []
chord_output = []

note_to_int_melody = {n: i for i, n in enumerate(sorted(set(melody_notes)))}
note_to_int_chord = {n: i for i, n in enumerate(sorted(set(chord_notes)))}

for i in range(len(melody_notes) - sequence_length):
    melody_seq = melody_notes[i:i+sequence_length]
    chord_seq = chord_notes[i:i+sequence_length]

    melody_input.append([note_to_int_melody[n] for n in melody_seq])
    chord_input.append([note_to_int_chord[n] for n in chord_seq])

    melody_output.append(note_to_int_melody[melody_notes[i + sequence_length]])
    chord_output.append(note_to_int_chord[chord_notes[i + sequence_length]])


**Build and Train Dual-Output LSTM Modelt**

**Get Vocabulary Size**

In [ ]:
n_vocab_melody = len(note_to_int_melody)
n_vocab_chord = len(note_to_int_chord)


**Reshape Input for LSTM**

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Reshape melody input
X_melody = np.reshape(melody_input, (len(melody_input), sequence_length, 1))
X_melody = X_melody / float(n_vocab_melody)

# Reshape chord input
X_chord = np.reshape(chord_input, (len(chord_input), sequence_length, 1))
X_chord = X_chord / float(n_vocab_chord)


**One-Hot Encode Outputs**

In [ ]:
y_melody = to_categorical(melody_output, num_classes=n_vocab_melody)
y_chord = to_categorical(chord_output, num_classes=n_vocab_chord)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense

# Shared Input
input_melody = Input(shape=(sequence_length, 1), name='melody_input')
input_chord = Input(shape=(sequence_length, 1), name='chord_input')

# LSTM for melody
x1 = LSTM(128, return_sequences=True)(input_melody)
x1 = Dropout(0.3)(x1)
x1 = LSTM(128)(x1)
x1 = Dense(len(note_to_int_melody), activation='softmax', name='melody_output')(x1)

# LSTM for chords
x2 = LSTM(128, return_sequences=True)(input_chord)
x2 = Dropout(0.3)(x2)
x2 = LSTM(128)(x2)
x2 = Dense(len(note_to_int_chord), activation='softmax', name='chord_output')(x2)

model = Model(inputs=[input_melody, input_chord], outputs=[x1, x2])
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(
    [X_melody, X_chord],
    [y_melody, y_chord],
    epochs=50,
    batch_size=64
)


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 194ms/step - chord_output_loss: 1.6736 - loss: 4.8464 - melody_output_loss: 3.1728
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 189ms/step - chord_output_loss: 1.1614 - loss: 4.0123 - melody_output_loss: 2.8510
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 12s 194ms/step - chord_output_loss: 1.1408 - loss: 3.9838 - melody_output_loss: 2.8430
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 258ms/step - chord_output_loss: 1.1686 - loss: 4.0210 - melody_output_loss: 2.8524
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 11s 174ms/step - chord_output_loss: 1.1404 - loss: 3.9746 - melody_output_loss: 2.8342
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step - chord_output_loss: 1.1816 - loss: 3.9874 - melody_output_loss: 2.8058
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 199ms/step - chord_output_loss: 1.1878 - loss: 3.9626 - melody_output_loss: 2.7748
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 248ms/step - chord_output_loss: 1.1574 - loss: 3.9568 - melody_output_loss: 2.7993


**Generate Both Melody and Harmony**

In [ ]:
import random

random_index = random.randint(0, len(melody_input) - 1)


In [ ]:
int_to_note_melody = {i: n for n, i in note_to_int_melody.items()}
int_to_note_chord = {i: n for n, i in note_to_int_chord.items()}


In [ ]:
generated_melody = []
generated_chords = []

pattern_melody = melody_input[random_index]
pattern_chord = chord_input[random_index]

for _ in range(100):
    pred_m, pred_c = model.predict(
        [np.reshape(pattern_melody, (1, sequence_length, 1)) / float(n_vocab_melody),
         np.reshape(pattern_chord, (1, sequence_length, 1)) / float(n_vocab_chord)]
    )
    note_m = int_to_note_melody[np.argmax(pred_m)]
    note_c = int_to_note_chord[np.argmax(pred_c)]

    generated_melody.append(note_m)
    generated_chords.append(note_c)

    pattern_melody.append(note_to_int_melody[note_m])
    pattern_chord.append(note_to_int_chord[note_c])

    pattern_melody = pattern_melody[1:]
    pattern_chord = pattern_chord[1:]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━

**Create Multi-Track MIDI Filet**

In [ ]:
from music21 import stream, note, chord, instrument

melody_stream = stream.Part()
melody_stream.insert(0, instrument.Piano())  # or another instrument

chord_stream = stream.Part()
chord_stream.insert(0, instrument.StringInstrument())  # fixed instrument

for i in range(len(generated_melody)):
    # Melody notes
    if generated_melody[i] != "None":
        n = note.Note(generated_melody[i])
        n.offset = i
        n.storedInstrument = instrument.Piano()
        melody_stream.append(n)

    # Chord notes
    if generated_chords[i] != "None":
        chord_notes = [int(x) for x in generated_chords[i].split('.')]
        c = chord.Chord(chord_notes)
        c.offset = i
        c.storedInstrument = instrument.StringInstrument()
        chord_stream.append(c)

# Combine both parts into a score
score = stream.Score([melody_stream, chord_stream])
score.write('midi', fp='multi_track_output.mid')


'multi_track_output.mid'

In [ ]:
!pip install pretty_midi
!apt-get install -y fluidsynth


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fluidsynth is already the newest version (2.2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!wget -O "FluidR3_GM.sf2" https://github.com/urish/cinto/raw/master/examples/soundfonts/FluidR3_GM.sf2


--2025-07-10 18:50:18--  https://github.com/urish/cinto/raw/master/examples/soundfonts/FluidR3_GM.sf2
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-10 18:50:19 ERROR 404: Not Found.



In [ ]:
import os

# File names
midi_file = "multi_track_output.mid"
soundfont_file = "FluidR3_GM.sf2"
output_wav = "generated_music.wav"

# Convert MIDI to WAV
!fluidsynth -ni {soundfont_file} {midi_file} -F {output_wav} -r 44100


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: error: fluid_is_soundfont(): failed to read RIFF chunk id.
Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.
Rendering audio to file 'generated_music.wav'..


In [ ]:
from google.colab import files
files.download(output_wav)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Conversion to lufi song**

**Tu Jaane Naa Lufi version**

In [ ]:
!pip install pydub librosa noisereduce
!apt-get install ffmpeg -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload 'Tu Jaane Na.mp3'


Saving Tu Jaane Na-320kbps.mp3 to Tu Jaane Na-320kbps.mp3


In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_file("Tu Jaane Na-320kbps.mp3")
audio.export("tujaanana.wav", format="wav")


<_io.BufferedRandom name='tujaanana.wav'>

In [ ]:
import librosa
import soundfile as sf

# Load audio
y, sr = librosa.load("tujaanana.wav", sr=None)

# Slow it down by 20%
y_slow = librosa.effects.time_stretch(y, rate=0.8)

# Lower pitch by 2 semitones
y_lofi = librosa.effects.pitch_shift(y_slow, sr=sr, n_steps=-2)

# Save intermediate output
sf.write("lofi_base.wav", y_lofi, sr)


In [ ]:
from pydub.generators import WhiteNoise
from pydub import AudioSegment, effects

# Load the processed track
song = AudioSegment.from_wav("lofi_base.wav")

# Add vinyl crackle noise
crackle = WhiteNoise().to_audio_segment(duration=len(song), volume=-35)
with_crackle = song.overlay(crackle)

# Apply low-pass filter to soften highs
lofi_filtered = with_crackle.low_pass_filter(3500)

# Normalize for volume balance
lofi_final = effects.normalize(lofi_filtered)

# Export final version
lofi_final.export("tujaanana_lofi.wav", format="wav")


<_io.BufferedRandom name='tujaanana_lofi.wav'>

In [ ]:
files.download("tujaanana_lofi.wav")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Adding background rain in Lufi(Tu Jaane Naa)**

In [ ]:
from google.colab import files
uploaded = files.upload()  # select your downloaded rain MP3


Saving sleepy-rain-116521.mp3 to sleepy-rain-116521.mp3


In [ ]:
from pydub import AudioSegment

lofi = AudioSegment.from_wav("tujaanana_lofi.wav")
rain = AudioSegment.from_file(list(uploaded.keys())[0])  # load your rain MP3

# Loop rain up to the Lofi duration
while len(rain) < len(lofi):
    rain += rain
rain = rain[:len(lofi)]

rain = rain - 10  # reduce rain volume
lofi_with_rain = lofi.overlay(rain)

lofi_with_rain.export("tujaanana_lofi_rain.wav", format="wav")


<_io.BufferedRandom name='tujaanana_lofi_rain.wav'>

In [ ]:
from google.colab import files
files.download("tujaanana_lofi_rain.wav")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip freeze > requirements.txt


In [2]:
from google.colab import files
files.download("requirements.txt")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>